Segment Anything segmented Pano-image to Point Cloud

In [8]:
!pip install laspy rasterio plyfile trimesh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 712.0/712.0 kB 10.3 MB/s eta 0:00:00


In [10]:
import laspy
import numpy as np
import rasterio
import cv2
import trimesh

# === Parameters ===
INPUT_LAS = r"2025_05_22_AF_RIEGL_01.las"
MASK_TIF = r"/content/labeled_mask.tif"
OUTPUT_PLY = r"output_with_mask.ply"
IMAGE_HEIGHT = 4096
IMAGE_WIDTH = 8192
PRECISION = 16
FACTOR = 2 ** PRECISION

# === Load mask ===
print("[1] Loading mask...")
with rasterio.open(MASK_TIF) as src:
    mask = src.read(1)

if mask.shape != (IMAGE_HEIGHT, IMAGE_WIDTH):
    print("[2] Resizing mask...")
    mask = cv2.resize(mask, (IMAGE_WIDTH, IMAGE_HEIGHT), interpolation=cv2.INTER_NEAREST)

# === Load LAS ===
print("[3] Reading LAS...")
las = laspy.read(INPUT_LAS)
x, y, z = las.x, las.y, las.z
r = np.sqrt(x * x + y * y + z * z)

# === Spherical projection for mask sampling ===
theta = ((IMAGE_WIDTH * FACTOR) - (((np.arctan2(y, x) + np.pi) / (2 * np.pi)) * IMAGE_WIDTH * FACTOR)).astype(np.uint64)
phi = ((np.arccos(z / r) / np.pi) * IMAGE_HEIGHT * FACTOR).astype(np.uint64)
theta_img = (theta >> PRECISION).clip(0, IMAGE_WIDTH - 1)
phi_img = (phi >> PRECISION).clip(0, IMAGE_HEIGHT - 1)

# === Sample mask ===
print("[4] Sampling mask...")
mask_label = mask[phi_img, theta_img].astype(np.uint8)

# === Extract fields and stack ===
print("[5] Stacking data...")
red = las.red.astype(np.uint16)
green = las.green.astype(np.uint16)
blue = las.blue.astype(np.uint16)
# Include mask_label as classification
output_data = np.column_stack((x, y, z, red, green, blue, mask_label))

# === Save to PLY ===
print("[6] Writing to PLY...")
# Create a mesh using trimesh
points = output_data[:, :3]  # x, y, z
colors = output_data[:, 3:6]  # r, g, b
classification = output_data[:, 6]  # mask label for classification

# Create a PLY mesh with classification
mesh = trimesh.Trimesh(vertices=points, vertex_colors=colors, process=False)

# Add classification as a vertex property
mesh.visual.vertex_colors = np.hstack((colors, classification[:, np.newaxis]))

# Save the mesh to a PLY file
mesh.export(OUTPUT_PLY)
print(f"[7] Done. Saved to {OUTPUT_PLY}")

[1] Loading mask...
[2] Resizing mask...
[3] Reading LAS...
[4] Sampling mask...
[5] Stacking data...
[6] Writing to PLY...
[7] Done. Saved to output_with_mask.ply
